In [1]:
(in-package "SMT")
(include-book "util")

 "SMT"

ACL2 Warning [Compiled file] in ( INCLUDE-BOOK "util" ...):  Unable
to load compiled file for book
  /workspaces/acl2-swf-experiments/experiments/smtlink/examples/util.lisp
because that book is not certified.  See :DOC include-book.  No load
was in progress for any parent book.


ACL2 Warning [Uncertified] in ( INCLUDE-BOOK "util" ...):  There is
no certificate on file for 
"/workspaces/acl2-swf-experiments/experiments/smtlink/examples/util.lisp".
See :DOC uncertified-books.


TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

TTAG NOTE (for included book): Adding ttag :QUICKLISP.SHELLPOOL from book /home/acl2/books/quicklisp/shellpool.

TTAG NOTE (for included book): Adding ttag :TSHELL from book /home/acl2/books/projects/smtlink/trusted/run.

TTAG NOTE (for included book): Adding ttag :TSHELL from book /home/acl2/books/projects/smtlink/trusted/write.

TTAG NOTE (for included book): Adding ttag :TSHELL from book /home/acl2/books/p

In [2]:
(defprod inverter
  ((input sig-path-p)
   (output sig-path-p)))

Form:  ( TABLE ACL2::XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.

In [3]:
(define inverter-sigs ((i inverter-p))
  :returns (l sig-path-listp)
  (b* ((i (inverter-fix i))
       (input (inverter->input i))
       (output (inverter->output i)))
    (cons input (sig-path-list-fix (cons output (sig-path-list-fix nil))))))


Since INVERTER-SIGS is non-recursive, its admission is trivial.  We
observe that the type of INVERTER-SIGS is described by the theorem
(AND (CONSP (INVERTER-SIGS I)) (TRUE-LISTP (INVERTER-SIGS I))).  We
used primitive type reasoning and the :type-prescription rule 
SIG-PATH-LIST-FIX$INLINE.

Computing the guard conjecture for INVERTER-SIGS....

The non-trivial part of the guard conjecture for INVERTER-SIGS, given
the :executable-counterparts of SIG-PATH-LIST-FIX$INLINE and SIG-PATH-LISTP,
is

Goal
(IMPLIES (INVERTER-P I)
         (AND (INVERTER-P I)
              (LET ((I (INVERTER-FIX I)))
                (AND (INVERTER-P I)
                     (INVERTER-P I)
                     (LET ((OUTPUT (INVERTER->OUTPUT I)))
                       (SIG-PATH-LISTP (LIST OUTPUT))))))).
Goal'

Q.E.D.

That completes the proof of the guard theorem for INVERTER-SIGS.  
INVERTER-SIGS is compliant with Common Lisp.

Summary
Form:  ( DEFUN INVERTER-SIGS ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-

In [4]:
(define inverter-count ((i inverter-p)
                        (curr any-table-p))
  :returns (markers maybe-integer-p)
  :guard-hints (("Goal" :in-theory (e/d (sigs-in-bool-table inverter-sigs))))
  (b* ((i (inverter-fix i))
       (curr (any-table-fix curr))
       ((unless (sigs-in-bool-table (inverter-sigs i) curr))
        (maybe-integer-fix nil))
       (in-sig (inverter->input i))
       (input (cdr (smt::magic-fix 'sig-path_booleanp
                                   (assoc-equal in-sig (any-table-fix curr)))))
       (out-sig (inverter->output i))
       (output (cdr (smt::magic-fix 'sig-path_booleanp
                                    (assoc-equal out-sig (any-table-fix curr))))))
    (maybe-integer-some (boolval (equal input output))))
  )


Since INVERTER-COUNT is non-recursive, its admission is trivial.  We
observe that the type of INVERTER-COUNT is described by the theorem
(OR (INTEGERP (INVERTER-COUNT I CURR)) (EQUAL (INVERTER-COUNT I CURR) NIL)).
We used the :type-prescription rules MAYBE-INTEGER-FIX$INLINE and 
MAYBE-INTEGER-SOME.

Computing the guard conjecture for INVERTER-COUNT....

The non-trivial part of the guard conjecture for INVERTER-COUNT, given
the :executable-counterpart of MAYBE-INTEGER-P, is

Goal
(IMPLIES
 (AND (ANY-TABLE-P CURR) (INVERTER-P I))
 (AND
  (INVERTER-P I)
  (LET ((I (INVERTER-FIX I)))
   (AND
    (ANY-TABLE-P CURR)
    (LET ((CURR (ANY-TABLE-FIX CURR)))
     (AND
      (INVERTER-P I)
      (SIG-PATH-LISTP (INVERTER-SIGS I))
      (ANY-TABLE-P CURR)
      (OR (NOT (SIGS-IN-BOOL-TABLE (INVERTER-SIGS I)
                                   CURR))
          (INVERTER-P I))
      (OR
       (NOT (SIGS-IN-BOOL-TABLE (INVERTER-SIGS I)
                                CURR))
       (LET ((IN-SIG (IN

In [5]:
;; This definition allows stuttering
(define inverter-valid-step ((i inverter-p)
                             (prev any-table-p)
                             (next any-table-p))
  :returns (ok booleanp)
  :guard-hints (("Goal" :in-theory (e/d (sigs-in-bool-table inverter-sigs))))
  (b* ((i (inverter-fix i))
       (prev (any-table-fix prev))
       (next (any-table-fix next))
       ((unless (sigs-in-bool-table (inverter-sigs i) prev)) nil)
       ((unless (sigs-in-bool-table (inverter-sigs i) next)) nil)
       (input (inverter->input i))
       (output (inverter->output i))
       (in-prev (cdr (smt::magic-fix 'sig-path_booleanp
                                     (assoc-equal input (any-table-fix prev)))))
       (out-prev (cdr (smt::magic-fix 'sig-path_booleanp
                                      (assoc-equal output (any-table-fix prev)))))
       (in-next (cdr (smt::magic-fix 'sig-path_booleanp
                                     (assoc-equal input (any-table-fix next)))))
       (out-next (cdr (smt::magic-fix 'sig-path_booleanp
                                      (assoc-equal output (any-table-fix next))))))
    (cond
     ;; if in-prev != out-prev
     ;; and out-next != out-prev
     ((and (not (equal in-prev out-prev))
           (not (equal out-next out-prev)))
      nil)
     ;; if in-prev == out-prev
     ;; and in-prev != in-next
     ((and (equal in-prev out-prev)
           (not (equal in-prev in-next)))
      t)
     ;; all other cases are t
     (t t)))
  )


Since INVERTER-VALID-STEP is non-recursive, its admission is trivial.
We observe that the type of INVERTER-VALID-STEP is described by the
theorem 
(OR (EQUAL (INVERTER-VALID-STEP I PREV NEXT)
           T)
    (EQUAL (INVERTER-VALID-STEP I PREV NEXT)
           NIL)).
We used primitive type reasoning.

Computing the guard conjecture for INVERTER-VALID-STEP....

The non-trivial part of the guard conjecture for INVERTER-VALID-STEP
is

Goal
(IMPLIES
 (AND (ANY-TABLE-P NEXT)
      (ANY-TABLE-P PREV)
      (INVERTER-P I))
 (AND
  (INVERTER-P I)
  (LET ((I (INVERTER-FIX I)))
   (AND
    (ANY-TABLE-P PREV)
    (LET ((PREV (ANY-TABLE-FIX PREV)))
     (AND
      (ANY-TABLE-P NEXT)
      (LET ((NEXT (ANY-TABLE-FIX NEXT)))
       (AND
        (INVERTER-P I)
        (SIG-PATH-LISTP (INVERTER-SIGS I))
        (ANY-TABLE-P PREV)
        (OR (NOT (SIGS-IN-BOOL-TABLE (INVERTER-SIGS I)
                                     PREV))
            (INVERTER-P I))
        (OR (NOT (SIGS-IN-BOOL-TABLE (INVERTE

In [6]:
(define inverter-valid ((i inverter-p)
                        (tr any-trace-p))
  :returns (ok booleanp)
  :measure (len tr)
  (b* ((i (inverter-fix i))
       ((unless (consp (any-trace-fix (cdr (any-trace-fix tr))))) t)
       (first (car (any-trace-fix tr)))
       (rest (cdr (any-trace-fix tr)))
       (second (car (any-trace-fix rest)))
       ((unless (inverter-valid-step i first second)) nil))
    (inverter-valid i rest))
  )


For the admission of INVERTER-VALID we will use the relation O< (which
is known to be well-founded on the domain recognized by O-P) and the
measure (LEN TR).  The non-trivial part of the measure conjecture is

Goal
(AND
 (O-P (LEN TR))
 (IMPLIES
   (AND (CONSP (ANY-TRACE-FIX (CDR (ANY-TRACE-FIX TR))))
        (INVERTER-VALID-STEP (INVERTER-FIX I)
                             (CAR (ANY-TRACE-FIX TR))
                             (CAR (ANY-TRACE-FIX (CDR (ANY-TRACE-FIX TR))))))
   (O< (LEN (CDR (ANY-TRACE-FIX TR)))
       (LEN TR)))).
Goal'

Splitter note (see :DOC splitter) for Goal' (2 subgoals).
  if-intro: ((:DEFINITION LEN))

Subgoal 2
Subgoal 2'
Subgoal 2''
Subgoal 1

Q.E.D.

That completes the proof of the measure theorem for INVERTER-VALID.
Thus, we admit this function under the principle of definition.  We
observe that the type of INVERTER-VALID is described by the theorem
(OR (EQUAL (INVERTER-VALID I TR) T) (EQUAL (INVERTER-VALID I TR) NIL)).

Computing the guard conjecture fo